In [1]:
import torch
import bittensor as bt
import os.path
from multiprocessing import Pool
import pandas as pd
import numpy as np
from substrateinterface import SubstrateInterface, Keypair

subtensor = bt.subtensor('archive')

In [2]:
subtensor.block

4144168

## Global param

In [4]:
foundation_hk = '5F4tQyWrhfGVcNhoqeiNsR6KjD4wMZ2kfhLj4oHYuyHbZAc3'
corcel_hk = '5GKH9FPPnWSUoeeTJp19wVtd84XqFW4pyK2ijV2GsFbhTrP1'
# corcel_hk = '5DQ2Geab6G25wiZ4jGH6wJM8fekrm1QhV9hrRuntjBVxxKZm'

default_foundation_stake = 1237815

netuids = range(0, 52)
block = 4144168

metagraph_storage_path = "./metagraphs"
processes = 32

## Medullar performance 

In [5]:
# download updated metagraphs
metas = {}
for netuid in range(52):
    metas[netuid] = subtensor.metagraph(netuid = netuid, lite = True)
    file_name = f"{metagraph_storage_path}/netuid{netuid}_block{block}.pt"
    torch.save(metas[netuid], file_name)

In [6]:
# load metagraphs
metas = {}
for netuid in range(52):
    file_name = f"{metagraph_storage_path}/netuid{netuid}_block{block}.pt"
    metas[netuid] = torch.load(file_name)
    print(metas[netuid])

/tmp/ipykernel_2219619/1800645493.py:5: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  metas[netuid] = torch.load(file_name)


metagraph(netuid:0, n:64, block:4144172, network:archive)
metagraph(netuid:1, n:1024, block:4144172, network:archive)
metagraph(netuid:2, n:256, block:4144172, network:archive)
metagraph(netuid:3, n:256, block:4144172, network:archive)
metagraph(netuid:4, n:256, block:4144172, network:archive)
metagraph(netuid:5, n:256, block:4144172, network:archive)
metagraph(netuid:6, n:256, block:4144172, network:archive)
metagraph(netuid:7, n:256, block:4144172, network:archive)
metagraph(netuid:8, n:256, block:4144172, network:archive)
metagraph(netuid:9, n:256, block:4144172, network:archive)
metagraph(netuid:10, n:256, block:4144172, network:archive)
metagraph(netuid:11, n:256, block:4144172, network:archive)
metagraph(netuid:12, n:256, block:4144172, network:archive)
metagraph(netuid:13, n:256, block:4144172, network:archive)
metagraph(netuid:14, n:256, block:4144172, network:archive)
metagraph(netuid:15, n:256, block:4144172, network:archive)
metagraph(netuid:16, n:256, block:4144172, network

In [7]:
substrate = SubstrateInterface(
    url="wss://archive.chain.opentensor.ai:443/",
    ss58_format=42,
    type_registry_preset='legacy'
)

block_hash = substrate.get_block_hash(block)
tempos = substrate.query_map("SubtensorModule", "Tempo", block_hash=block_hash)

# === Live emission
def get_subnet_emission(block):
    emissions = np.array([])
    for netuid in range(52):
        meta = metas[netuid]
        # print( f"{netuid}, {meta.E.sum():.3f}, {E[netuid].item() * tempos[netuid][1].value * 0.82:.3f}")
        emissions = np.append(emissions, meta.E.sum() / (tempos[netuid][1].value * 0.82))
    return torch.tensor(emissions)

E = get_subnet_emission(None)
E.sum()

tensor(0.8050, dtype=torch.float64)

In [8]:
df = []
for netuid in range(1, 52):
    meta = metas[netuid]
    existing_stake = meta.stake[meta.validator_trust > 0].sum()
    foundation_uid = meta.hotkeys.index(foundation_hk) if foundation_hk in meta.hotkeys else None
    foundation_stake = meta.stake[foundation_uid] if foundation_hk in meta.hotkeys else None
    subnet_emission = E[netuid].item()

    if foundation_uid is None: 
        expected_foundation_dividend = default_foundation_stake / (default_foundation_stake + existing_stake)
        foundation_dividend = None
    else:
        expected_foundation_dividend = foundation_stake / existing_stake
        foundation_dividend = meta.D[foundation_uid]

    expected_foundation_emission = expected_foundation_dividend * subnet_emission * 0.82 * 0.5

    df.append({
        'netuid': netuid,
        'stake': existing_stake,
        'foundation_uid': foundation_uid,
        'subnet_emission': subnet_emission,
        'foundation_dividend': foundation_dividend,
        'expected_foundation_dividend': expected_foundation_dividend,
        'expected_foundation_emission': expected_foundation_emission
    })

df = pd.DataFrame(df)
df['boosted_APY'] = df.expected_foundation_emission / df[~df.foundation_uid.isna()].expected_foundation_emission.sum() + 1
df[df.foundation_uid.isna()].sort_values('expected_foundation_emission', ascending = False)

,netuid,stake,foundation_uid,subnet_emission,foundation_dividend,expected_foundation_dividend,expected_foundation_emission,boosted_APY
3,4,5906627.5,NaN,4.698920e-02,NaN,0.173256,3.337869e-03,1.066675
36,37,4821927.0,NaN,1.333356e-02,NaN,0.204269,1.116687e-03,1.022306
50,51,5092082.0,NaN,1.119009e-02,NaN,0.195551,8.971734e-04,1.017921
35,36,0.0,NaN,2.456300e-07,NaN,1.000000,1.007083e-07,1.000002
46,47,0.0,NaN,0.000000e+00,NaN,1.000000,0.000000e+00,1.000000


## Medullar performance compared to corcel

In [9]:
def _download_metagraph(netuid, block, file_name):
    if os.path.isfile(file_name):
        return
    else:
        print(netuid, block, file_name, " | downloading...")
    archive_subtensor = bt.subtensor("archive")
    meta = archive_subtensor.metagraph(netuid=netuid, block=block, lite=False)
    torch.save(meta, file_name)
    return


def download_metagraph():
    if not os.path.isdir(metagraph_storage_path):
        os.mkdir(metagraph_storage_path)

    args = []
    for netuid in netuids:
        file_name = f"{metagraph_storage_path}/netuid{netuid}_block{block}.pt"
        args.append((netuid, block, file_name))
    args = set(args)
    with Pool(processes=processes) as pool:
        pool.starmap(_download_metagraph, args)

download_metagraph()

In [10]:
def get_metagraphs():
    def load_metagraph(file_name):
        try:
            meta = torch.load(file_name)
            return meta
        except:
            return None

    metas = {}
    
    # === Collect metagraphs that has to be loaded ===
    for netuid in netuids:
        file_name = f"{metagraph_storage_path}/netuid{netuid}_block{block}.pt"
        metas[netuid] = load_metagraph(file_name)

    return metas

metas = get_metagraphs()

/tmp/ipykernel_2219619/1148291225.py:4: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  meta = torch.load(file_name)


In [62]:
meta = metas[1]

sum(meta.E )/0.82 / tempos[1][1].value

np.float32(0.041106027)

In [67]:
df = []
for netuid in netuids:
    if netuid == 0 or netuid == 36:
        continue
    meta = metas[netuid]
    vt = meta.validator_trust[(meta.validator_trust > 0) & (meta.validator_trust < 0.98)]
    vt.sort()
    if len(vt) > 0:
        top_vt = vt[-1]
    else:
        top_vt = None
    
    if foundation_hk in meta.hotkeys:
        uid = meta.hotkeys.index(foundation_hk)
        foundation_vt = meta.validator_trust[uid] 
        foundation_e = meta.E[uid]/ meta.S[uid] / tempos[netuid][1].value  # emission per stake per block
        foundation_daily_e = meta.E[uid] / tempos[netuid][1].value * 7200 # including to norminators
        foundation_uid = uid
    else:
        foundation_vt = None
        foundation_e = None
        foundation_uid = None
        
    if corcel_hk in meta.hotkeys:
        uid = meta.hotkeys.index(corcel_hk)
        corcel_vt = meta.validator_trust[uid] # / meta.S[uid]* 1_000_000 
        corcel_e = meta.E[uid]/ meta.S[uid] / tempos[netuid][1].value  # emission per stake per block
    
    else:    
        corcel_vt = None
        corcel_e = None
    
    df.append({
        "netuid": netuid,
        "emission": E[netuid].item(), 
        "top_vt": top_vt,
        "foundation_uid": foundation_uid,
        "foundation_vt": foundation_vt,
        "corcel_vt": corcel_vt,
        "foundation_e": foundation_e,
        "corcel_e": corcel_e,
        "foundation_daily_e": foundation_daily_e,
    })

df = pd.DataFrame(df)
df['under_performing_e'] = df['corcel_e'] - df['foundation_e']  
df['margin'] =  df['under_performing_e'] / df['foundation_e']

synapse_take = 0.08
df['foundation_daily_income_org'] = df.foundation_daily_e * 0.18
df['foundation_daily_income'] = df.foundation_daily_e * (1 + df.margin) * (1 - synapse_take) * 0.18
df['synapse_daily_income'] = df.foundation_daily_e * (1 + df.margin) * synapse_take
df['synapse_take'] = synapse_take
df = df[(df.under_performing_e > 0)].sort_values(by='margin', ascending=False)

In [68]:
_df = df[(df.under_performing_e > 0)].sort_values(by='margin', ascending=False)

In [78]:
_df

,netuid,emission,top_vt,foundation_uid,foundation_vt,corcel_vt,foundation_e,corcel_e,foundation_daily_e,under_performing_e,margin,foundation_daily_income_org,foundation_daily_income,synapse_daily_income,synapse_take
36,38,0.003266,0.974945,64.0,0.295857,0.925200,1.008202e-10,3.117165e-10,0.572696,2.108962e-10,2.091805,0.103085,0.293222,0.141653,0.08
43,45,0.010190,0.961746,250.0,0.708675,0.925856,5.825231e-10,8.615811e-10,4.024038,2.790580e-10,0.479051,0.724327,0.985611,0.476140,0.08
24,25,0.025236,0.956893,143.0,0.559091,0.943313,1.468365e-09,2.149144e-09,7.414095,6.807788e-10,0.463630,1.334537,1.797008,0.868120,0.08
28,29,0.040539,0.914199,143.0,0.614542,0.799878,2.255653e-09,3.115420e-09,12.812943,8.597678e-10,0.381161,2.306330,2.930581,1.415739,0.08
2,3,0.003330,0.902480,154.0,0.682902,0.883757,1.883929e-10,2.432410e-10,1.420313,5.484806e-11,0.291136,0.255656,0.303680,0.146705,0.08
40,42,0.004251,0.956603,255.0,0.809186,0.988922,3.005030e-10,3.707875e-10,2.265521,7.028458e-11,0.233890,0.407794,0.462919,0.223632,0.08
20,21,0.014235,0.958328,224.0,0.780377,0.925704,1.002608e-09,1.208312e-09,5.695185,2.057040e-10,0.205169,1.025133,1.136622,0.549093,0.08
44,46,0.000321,0.955749,70.0,0.864988,0.992187,3.311493e-11,3.851283e-11,0.209006,5.397904e-12,0.163005,0.037621,0.040253,0.019446,0.08
7,8,0.024190,0.960983,69.0,0.819043,0.922560,1.518454e-09,1.705896e-09,8.625381,1.874423e-10,0.123443,1.552569,1.604684,0.775210,0.08
27,28,0.015041,0.972351,217.0,0.853132,0.972351,1.040282e-09,1.157399e-09,5.909189,1.171166e-10,0.112582,1.063654,1.088730,0.525956,0.08


In [77]:
sum(df.foundation_e) * 7200 # emission per stake in a year


0.00015640988307030756

In [73]:
sum(df.corcel_e) * 7200 * 365 # emission per stake in a year

0.06509744121283717

In [76]:
1 - 0.057 / 0.065

0.12307692307692308

## Medullar child graph

In [31]:
substrate = SubstrateInterface(url="wss://entrypoint-finney.opentensor.ai:443")
parent = foundation_hk

results = {}
for netuid in netuids:
    if netuid == 0:
        continue
    try: 
        result = substrate.query('SubtensorModule', 'ChildKeys', [parent, netuid]).value[0]
        results[netuid] = [result[0]/2**64, result[1]]
    except:
        continue

In [32]:
results

{1: [0.1, '5DQ2Geab6G25wiZ4jGH6wJM8fekrm1QhV9hrRuntjBVxxKZm'],
 2: [0.1, '5DQ2Geab6G25wiZ4jGH6wJM8fekrm1QhV9hrRuntjBVxxKZm'],
 3: [0.1, '5DQ2Geab6G25wiZ4jGH6wJM8fekrm1QhV9hrRuntjBVxxKZm'],
 4: [0.1, '5DQ2Geab6G25wiZ4jGH6wJM8fekrm1QhV9hrRuntjBVxxKZm'],
 5: [0.1, '5DQ2Geab6G25wiZ4jGH6wJM8fekrm1QhV9hrRuntjBVxxKZm'],
 6: [0.1, '5DQ2Geab6G25wiZ4jGH6wJM8fekrm1QhV9hrRuntjBVxxKZm'],
 7: [0.1, '5DQ2Geab6G25wiZ4jGH6wJM8fekrm1QhV9hrRuntjBVxxKZm'],
 8: [0.1, '5DQ2Geab6G25wiZ4jGH6wJM8fekrm1QhV9hrRuntjBVxxKZm'],
 9: [0.1, '5DQ2Geab6G25wiZ4jGH6wJM8fekrm1QhV9hrRuntjBVxxKZm'],
 10: [0.1, '5DQ2Geab6G25wiZ4jGH6wJM8fekrm1QhV9hrRuntjBVxxKZm'],
 11: [0.1, '5DQ2Geab6G25wiZ4jGH6wJM8fekrm1QhV9hrRuntjBVxxKZm'],
 12: [0.1, '5DQ2Geab6G25wiZ4jGH6wJM8fekrm1QhV9hrRuntjBVxxKZm'],
 13: [0.1, '5DQ2Geab6G25wiZ4jGH6wJM8fekrm1QhV9hrRuntjBVxxKZm'],
 14: [0.1, '5DQ2Geab6G25wiZ4jGH6wJM8fekrm1QhV9hrRuntjBVxxKZm'],
 15: [0.1, '5DQ2Geab6G25wiZ4jGH6wJM8fekrm1QhV9hrRuntjBVxxKZm'],
 16: [0.1, '5DQ2Geab6G25wiZ4jGH6wJM8fekrm1QhV9hrR